In [1]:
%pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=fef3d7c2449294416cafa7fd14df0e2455274b3c4b4c0f36d29f31cb744fa611
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [2]:
!pip install torch
import torch

device = torch.device("cuda")

In [3]:
from spuco.utils import set_seed

set_seed(0)

In [4]:
from spuco.robust_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995, # define how many datas in a group will have spurious feature
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


100%|██████████| 9912422/9912422 [00:00<00:00, 51899768.99it/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 65161750.31it/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 35295965.07it/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17429578.01it/s]


Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 10000/10000 [00:01<00:00, 5340.10it/s]


In [5]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


100%|██████████| 11996/11996 [00:03<00:00, 3309.74it/s]


## Group-Balancing

In [21]:
# Use package spuco
from spuco.group_inference import JTTInference
from spuco.utils import Trainer
from spuco.robust_train import UpSampleERM, CustomSampleERM
import random
import numpy as np
from spuco.utils.random_seed import seed_randomness
from spuco.evaluate import Evaluator
from torch.optim import SGD
from spuco.models import model_factory

# Use JTT Inference after each epoch, and upsample it to 3 times of the max group

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)

print("Train before JTT Inference")
pre_infer_trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
pre_infer_trainer.train(1)

# Do JTT Inference
predictions = torch.argmax(pre_infer_trainer.get_trainset_outputs(), dim=-1).detach().cpu().tolist()
jtt_partition = JTTInference(
    predictions=predictions,
    class_labels=trainset.labels
).infer_groups()

# print("jtt_partition:", jtt_partition)
# Train with dataset after JTT Inference
val_evaluator = Evaluator(
    testset=valset,
    group_partition=valset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)

upsample_amount = 3

len_max_group = max([len(jtt_partition[key]) for key in jtt_partition.keys()])
print("len_max_group=", len_max_group)
print(jtt_partition.keys())
up_indices = []
for key in jtt_partition.keys():
    if key == (0, 1):
      up_indices.extend(jtt_partition[key]*int(len_max_group*upsample_amount/len(jtt_partition[key])))
    elif key == (0, 0):
      up_indices.extend(jtt_partition[key])
    else:
      print('len(jtt_partition[key]) <= 0! key=', key, ", len(jtt_partition[key])=", len(jtt_partition[key]))

print("\n Train after JTT Inference(1)")
post_infer_trainer = CustomSampleERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    val_evaluator=val_evaluator,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    indices=up_indices,
    device=device,
    verbose=True
)
post_infer_trainer.train()

len_max_group = max([len(jtt_partition[key]) for key in jtt_partition.keys()])
print(jtt_partition.keys())
up_indices = []
for key in jtt_partition.keys():
    if 0 < len(jtt_partition[key]) < len_max_group:
      up_indices.extend(jtt_partition[key]*int(len_max_group*upsample_amount/len(jtt_partition[key])))
    elif len(jtt_partition[key]) == len_max_group:
      up_indices.extend(jtt_partition[key])
    else:
      print('len(jtt_partition[key]) <= 0! key=', key, ", len(jtt_partition[key])=", len(jtt_partition[key]))

print("\n Train after JTT Inference(2)")
post_infer_trainer = CustomSampleERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    val_evaluator=val_evaluator,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    indices=up_indices,
    device=device,
    verbose=True
)
post_infer_trainer.train()

Train before JTT Inference


Getting Trainset Outputs: 100%|██████████| 751/751 [00:02<00:00, 304.79batch/s]


len_max_group= 47713
dict_keys([(0, 0), (0, 1)])

 Train after JTT Inference(1)


Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:06,  3.84it/s]

Group (0, 0) Accuracy: 99.80276134122288


...

Group (4, 2) Accuracy: 33.47457627118644


Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:07<00:00,  3.86it/s]

Group (4, 3) Accuracy: 59.53389830508475


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:07<00:00,  3.42it/s]

Group (4, 4) Accuracy: 100.0
Epoch 0: Val Worst-Group Accuracy: 13.333333333333334
Best Val Worst-Group Accuracy: 13.333333333333334


In [22]:
evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

evaluator.worst_group_accuracy

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:06,  3.63it/s]

Group (0, 0) Accuracy: 99.76359338061465


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:06,  3.75it/s]

Group (0, 1) Accuracy: 86.99763593380615


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:00<00:05,  3.81it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:07<00:00,  2.50it/s]

Group (4, 3) Accuracy: 56.06060606060606


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:07<00:00,  3.18it/s]

Group (4, 4) Accuracy: 99.74747474747475


((2, 4), 12.834224598930481)

#### Backup Code: Self-implimented JTT

In [17]:
# Self-implimented JTT
from spuco.group_inference import JTTInference
from spuco.utils import Trainer
from spuco.robust_train import UpSampleERM, CustomSampleERM
import random
import numpy as np
from spuco.utils.random_seed import seed_randomness
from spuco.evaluate import Evaluator
from torch.optim import SGD
from spuco.models import model_factory

# Use JTT Inference after each epoch, and upsample it to 3 times of the max group

model2 = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)

print("Train before JTT Inference")
pre_infer_trainer = Trainer(
    trainset=trainset,
    model=model2,
    batch_size=64,
    optimizer=SGD(model2.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
pre_infer_trainer.train(1)

# Do JTT Inference after first epoch
predictions = torch.argmax(pre_infer_trainer.get_trainset_outputs(), dim=-1).detach().cpu().tolist()
print("len(predictions) epoch 0:", len(predictions))

def myJTTUpsample(predictions, labels, upsample_amount: int=3):
  res = []
  mislabel = []
  match_key_len = 0
  unmatch_key_len = 0
  for idx, pred in enumerate(predictions):
    if pred == labels[idx]:
      res.append(idx)
      match_key_len += 1
    else:
      mislabel.append(idx)
      unmatch_key_len += 1

  res.extend(mislabel * int(upsample_amount*match_key_len/unmatch_key_len))
  random.shuffle(res)
  return res


up_indices = myJTTUpsample(predictions, trainset.labels, 3)
print(len(up_indices), "up_indices:", up_indices[:50])

val_evaluator = Evaluator(
    testset=valset,
    group_partition=valset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model2,
    device=device,
    verbose=True
)

print("\n Train after JTT Inference(1)")
post_infer_trainer = CustomSampleERM(
    model=model2,
    num_epochs=2,
    trainset=trainset,
    val_evaluator=val_evaluator,
    batch_size=64,
    optimizer=SGD(model2.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    indices=up_indices,
    device=device,
    verbose=True
)
post_infer_trainer.train()

# # Do JTT Inference after second epoch
# predictions = torch.argmax(pre_infer_trainer.get_trainset_outputs(), dim=-1).detach().cpu().tolist()
# up_indices = myJTTUpsample(predictions, trainset.labels, 10)

# print("\n Train after JTT Inference(2)")
# post_infer_trainer = CustomSampleERM(
#     model=model2,
#     num_epochs=1,
#     trainset=trainset,
#     val_evaluator=val_evaluator,
#     batch_size=64,
#     optimizer=SGD(model2.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
#     indices=up_indices,
#     device=device,
#     verbose=True
# )
# post_infer_trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Train before JTT Inference


Getting Trainset Outputs: 100%|██████████| 751/751 [00:02<00:00, 286.55batch/s]


len(predictions) epoch 0: 48004
190594 up_indices: [17460, 22433, 4901, 6524, 160, 29032, 33459, 33761, 36418, 7370, 22439, 13236, 36975, 47080, 11000, 747, 29009, 44156, 6002, 46902, 22120, 3079, 33840, 18188, 45195, 6045, 3934, 19429, 22221, 46902, 33189, 19132, 44780, 38116, 37675, 19118, 24102, 40997, 5986, 5986, 19645, 2978, 31116, 21186, 31982, 43999, 45297, 21025, 6991, 2042]

 Train after JTT Inference(1)


Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:09,  2.41it/s]

...

Group (4, 2) Accuracy: 32.41525423728814


Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:06<00:00,  3.89it/s]

Group (4, 3) Accuracy: 56.567796610169495


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:06<00:00,  3.68it/s]

Group (4, 4) Accuracy: 98.93842887473461
Epoch 1: Val Worst-Group Accuracy: 11.864406779661017
Best Val Worst-Group Accuracy: 19.915254237288135


In [20]:
evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model2,
    device=device,
    verbose=True
)
evaluator.evaluate()

evaluator.worst_group_accuracy

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:06,  3.94it/s]

Group (0, 0) Accuracy: 99.29078014184397


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:06,  3.81it/s]

Group (0, 1) Accuracy: 73.75886524822695


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:00<00:05,  3.78it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:06<00:00,  2.62it/s]

Group (4, 3) Accuracy: 57.323232323232325


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:07<00:00,  3.44it/s]

Group (4, 4) Accuracy: 99.4949494949495


((4, 1), 11.083123425692696)